## Appendix Table A4: Private equity excess IRR above S&P annual rate of return and partner trustee ties, without year fixed effects

### From Elite Embeddedness

By Charlie Eaton and Albina Gibadullina

In [1]:
use data/preqin_sandp_trustee_ties, clear

In [2]:
encode fund_type, gen(ifundtype)
encode fund_focus, gen(ifundfocus)

In [3]:
labelbook ifundtype


-------------------------------------------------------------------------------
Value label ifundtype 
-------------------------------------------------------------------------------

      Values                                    Labels
       Range:  [1,14]                    String length:  [6,27]
           N:  14                Unique at full length:  yes
        Gaps:  no                  Unique at length 12:  no
  Missing .*:  0                           Null string:  no
                               Leading/trailing blanks:  no
                                    Numeric -> numeric:  no
  Definition
           1   Balanced
           2   Buyout
           3   Co-Investment Multi-Manager
           4   Co-investment
           5   Direct Secondaries
           6   Early Stage
           7   Early Stage: Seed
           8   Early Stage: Start-up
           9   Expansion / Late Stage
          10   Fund of Funds
          11   Growth
          12   Secondaries
          13   Tu

In [4]:
replace ifundtype=6 if ifundtype==7 | ifundtype==9
replace ifundtype=3 if ifundtype==4
replace ifundtype=12 if ifundtype==5

(88 real changes made)
(51 real changes made)
(9 real changes made)


In [5]:
tab ifundtype


                  Fund_Type |      Freq.     Percent        Cum.
----------------------------+-----------------------------------
                   Balanced |         28        1.27        1.27
                     Buyout |        837       37.86       39.12
Co-Investment Multi-Manager |         92        4.16       43.28
                Early Stage |        224       10.13       53.41
      Early Stage: Start-up |         21        0.95       54.36
              Fund of Funds |        470       21.26       75.62
                     Growth |        187        8.46       84.08
                Secondaries |        122        5.52       89.60
                 Turnaround |          8        0.36       89.96
          Venture (General) |        222       10.04      100.00
----------------------------+-----------------------------------
                      Total |      2,211      100.00


In [6]:
gen net_irr_premium= net_irr_pcent-sp500_return_pct

In [7]:
labelbook ifundfocus


-------------------------------------------------------------------------------
Value label ifundfocus 
-------------------------------------------------------------------------------

      Values                                    Labels
       Range:  [1,8]                     String length:  [2,20]
           N:  8                 Unique at full length:  yes
        Gaps:  no                  Unique at length 12:  yes
  Missing .*:  0                           Null string:  no
                               Leading/trailing blanks:  no
                                    Numeric -> numeric:  no
  Definition
           1   Africa
           2   Americas
           3   Asia
           4   Australasia
           5   Diversified Multi-Re
           6   Europe
           7   Middle East & Israel
           8   US

   Variables:  ifundfocus



In [8]:
keep if ifundfocus>4 & ifundfocus!=7 

(270 observations deleted)


In [9]:
keep if fund_focus!=""

(7 observations deleted)


In [10]:
tab ifundfocus


          Fund_Focus |      Freq.     Percent        Cum.
---------------------+-----------------------------------
Diversified Multi-Re |         21        1.09        1.09
              Europe |        426       22.03       23.11
                  US |      1,487       76.89      100.00
---------------------+-----------------------------------
               Total |      1,934      100.00


In [11]:
xi i.ifundfocus, noomit prefix(_D)

In [12]:
ds _D*

_Difundfoc~5  _Difundfoc~6  _Difundfoc~8


In [13]:
xi i.fund_type, noomit

In [14]:
ds _I*

_Ifund_ty~_1  _Ifund_ty~_4  _Ifund_typ~7  _Ifund_ty~10  _Ifund_ty~13
_Ifund_ty~_2  _Ifund_typ~5  _Ifund_typ~8  _Ifund_ty~11  _Ifund_ty~14
_Ifund_ty~_3  _Ifund_typ~6  _Ifund_typ~9  _Ifund_ty~12


In [15]:
qui replace strength=0 if strength==.

In [16]:
collapse (max) rejectrt2017 (rawsum) fundvaluemnusd (first) firmname (min) public_board_ties private_board_ties ///
rejectrt otherfinance cpi_scalar_2017 rejectmax rejectrtmn rejectrtindex rejectprivate private public publictop30 privatetop30  ///
 betweenness strength eigenvector Edu_sci_med Non_finance Nonprofit Other_fin Alt_fin Public_sector Real_estate ///
(mean) _I* _D* net_irr_pcent net_irr_premium [aweight=fundvaluemnusd], by(firmid year)

In [17]:
qui gen lnfundvaluemnusd=log(fundvaluemnusd)

gen allother=strength-Non_finance_business

In [18]:
label var privatetop30 "private trustee seats"

label var publictop30 "public trustee seats"

label var rejectmax "Highest admissions selectivity of trustees"

label var lnfundvaluemnusd "Fund size (log)"

label var strength "Trustee network weighted degree"

label var Non_finance "Non-financial business ties"

label var  allother "All other trustee ties"

label var rejectrtindex "count of trustees weighted by admissions selectivity"

label var public_board_ties "public trustee ties"

label var private_board_ties "private trustee ties"

Note, the _I* and _D* variables are the proportions of firms funds under the fund types and fund focus regions reported above 

In [19]:
ds _I* _D*

_Ifund_ty~_1  _Ifund_typ~5  _Ifund_typ~9  _Ifund_ty~13  _Difundfoc~8
_Ifund_ty~_2  _Ifund_typ~6  _Ifund_ty~10  _Ifund_ty~14
_Ifund_ty~_3  _Ifund_typ~7  _Ifund_ty~11  _Difundfoc~5
_Ifund_ty~_4  _Ifund_typ~8  _Ifund_ty~12  _Difundfoc~6


In [20]:
set line 255
est clear
qui xtset firmid year

qui eststo: reghdfe net_irr_premium lnfundvaluemnusd privatetop30 publictop30 _I* _D*, absorb(firmid) cluster(firmid)

qui eststo: reghdfe net_irr_premium Non_finance_business ///
 lnfundvaluemnusd _I* _D*, absorb(firmid) cluster(firmid)

qui eststo: reghdfe net_irr_premium lnfundvaluemnusd ///
allother _I* _D*, absorb(firmid) cluster(firmid)

In [21]:
display "Table A4: Panel Fixed Effects Regression of Net Returns Premium over S&P"

esttab, type cells(b(star fmt(3)) se(fmt(3) par)) stardetach ///
	 legend label varlabels(_cons Constant) stats(r2 N N_clust, fmt(3 0 0) ///
	label(R-square N Firms)) keep(lnfundvaluemnusd ///
privatetop30 publictop30 allother ///
Non_finance_business ) ///
order(privatetop30 publictop30 private_board_ties public_board_ties ///
Non_finance_business allother) ///
starlevels(^ .1 * .05 ** .01 *** .001) mlabel(none) ///
collabels(none) eqlabels(none)

display "Note: All models include firm fixed effects."

Table A4: Panel Fixed Effects Regression of Net Returns Premium over S&P

-----------------------------------------------------------------------
                              (1)              (2)              (3)    
-----------------------------------------------------------------------
private trustee se~s        3.459 ***                                  
                          (0.929)                                      
public trustee seats       -2.529                                      
                          (6.132)                                      
Non-financial busi~s                         0.181 ***                 
                                           (0.051)                     
All other trustee ~s                                          0.114 ***
                                                            (0.032)    
Fund size (log)            -0.400           -0.377           -0.385    
                          (0.558)          (0.560)          (0

In [22]:
qui esttab using tables/ta4_returns_trustee_ties_firm_fe.rtf, type cells(b(star fmt(3)) se(fmt(3) par)) ///
	 legend label varlabels(_cons Constant) stats(r2 N N_clust, fmt(3 0 0) ///
	label(R-square N Firms)) keep(lnfundvaluemnusd ///
privatetop30 publictop30 allother ///
Non_finance_business ) ///
order(privatetop30 publictop30 rejectrtindex ///
Non_finance_business allother) ///
starlevels(^ .1 * .05 ** .01 *** .001) mlabel(none) ///
collabels(none) eqlabels(none) replace